In [31]:
import pandas as pd
import numpy as np
from datetime import datetime
from pydantic import BaseModel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

def house_prices_data():
    open_house_prices = pd.read_csv("../data/King_County_House_prices_dataset.csv")
    open_house_prices_data = open_house_prices.copy()
    return open_house_prices_data

class DataValidation(BaseModel):
    id: int
    date: datetime
    price: float 
    bedrooms: int 
    bathrooms: float  
    sqft_living: int 
    sqft_lot: int 
    floors: float
    waterfront: float
    view: float 
    condition: int 
    grade: int  
    sqft_above: int 
    sqft_basement: float
    zipcode: int
    lat: float 
    long: float  
    sqft_living15: int 
    sqft_lot15: int 
    last_known_change: int
    sqft_price: float
    delta_lat: float
    delta_long: float
    center_distance: float
    water_distance: float

def data_validation(df: pd.DataFrame, data_schema) -> pd.DataFrame:
    class DataframeValidation(BaseModel):
        df_as_dict: list[data_schema]
    df_as_dict = df.to_dict(orient='records')
    DataframeValidation(df_as_dict=df_as_dict)
    return df 

class removeOutlier(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X[(X['bedrooms'] / X['bathrooms']) <= 10]                                
        return X

#house_prices_sklearn = house_prices_data()
#remove_outlier = removeOutlier()
#house_prices_sklearn = remove_outlier.fit_transform(house_prices_sklearn)

class sqftBasementTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        X['sqft_basement'] = X['sqft_basement'].replace('?', np.NaN)
        X['sqft_basement'] = X['sqft_basement'].astype(float)
        X.eval('sqft_basement = sqft_living - sqft_above', inplace=True)                               
        return X

#house_prices_sklearn = house_prices_data()
#sqft_basement_transformer = sqftBasementTransformer()
#house_prices_sklearn = sqft_basement_transformer.fit_transform(house_prices_sklearn)

class viewTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['view'].fillna(0, inplace=True)                             
        return X

#house_prices_sklearn = house_prices_data()
#view_transformer = viewTransformer()
#house_prices_sklearn = view_transformer.fit_transform(house_prices_sklearn)

class waterfrontTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X.waterfront.fillna(0, inplace=True)                             
        return X

#house_prices_sklearn = house_prices_data()
#waterfront_transformer = waterfrontTransformer()
#house_prices_sklearn = waterfront_transformer.fit_transform(house_prices_sklearn)

class lastChangeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        last_known_change = []
        for idx, yr_re in X.yr_renovated.items():
            if str(yr_re) == 'nan' or yr_re == 0.0:
                last_known_change.append(X.yr_built[idx])
            else:
                last_known_change.append(int(yr_re))
        X['last_known_change'] = last_known_change
        X.drop('yr_renovated', axis=1, inplace=True)
        X.drop("yr_built", axis=1, inplace=True)
        return X

#house_prices_sklearn = house_prices_data()
#last_change_transformer = lastChangeTransformer()
#house_prices_sklearn = last_change_transformer.fit_transform(house_prices_sklearn)

# Summation of the missing values and calculation of the missing values as a percentage
#missing_values = pd.DataFrame(house_prices_sklearn.isnull().sum(),columns=['count'])
#missing_values['percentage'] = (missing_values['count']/house_prices_sklearn.shape[0]*100).round(2)
#missing_values.query('count != 0')

class dateTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['date'] = pd.to_datetime(X['date'])#, format='%Y/%m/%d').astype('object')                       
        return X

#house_prices_sklearn = house_prices_data()
#date_transformer = dateTransformer()
#house_prices_sklearn = date_transformer.fit_transform(house_prices_sklearn)

data_cleaning_pipeline = Pipeline([
    ('remove_outlier', removeOutlier()),
    ('sqft_basement_transformer', sqftBasementTransformer()),
    ('view_transformer', viewTransformer()),
    ('waterfront_transformer', waterfrontTransformer()),
    ('last_change_transformer', lastChangeTransformer()),
    ('date_transformer', dateTransformer())
     ]
)
house_prices_cl_pipeline = house_prices_data()
house_prices_cl_pipeline = data_cleaning_pipeline.fit_transform(house_prices_cl_pipeline)

house_prices_cl_pipeline.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21596 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 21596 non-null  int64         
 1   date               21596 non-null  datetime64[ns]
 2   price              21596 non-null  float64       
 3   bedrooms           21596 non-null  int64         
 4   bathrooms          21596 non-null  float64       
 5   sqft_living        21596 non-null  int64         
 6   sqft_lot           21596 non-null  int64         
 7   floors             21596 non-null  float64       
 8   waterfront         21596 non-null  float64       
 9   view               21596 non-null  float64       
 10  condition          21596 non-null  int64         
 11  grade              21596 non-null  int64         
 12  sqft_above         21596 non-null  int64         
 13  sqft_basement      21596 non-null  float64       
 14  zipcode    

In [32]:
# Summation of the missing values and calculation of the missing values as a percentage
missing_values = pd.DataFrame(house_prices_cl_pipeline.isnull().sum(),columns=['count'])
missing_values['percentage'] = (missing_values['count']/house_prices_cl_pipeline.shape[0]*100).round(2)
missing_values.query('count != 0')

,count,percentage


In [33]:
data_cleaning_pipeline

Pipeline(steps=[('remove_outlier', removeOutlier()),
                ('sqft_basement_transformer', sqftBasementTransformer()),
                ('view_transformer', viewTransformer()),
                ('waterfront_transformer', waterfrontTransformer()),
                ('last_change_transformer', lastChangeTransformer()),
                ('date_transformer', dateTransformer())])

In [34]:

def distance_calculater(long, lat, ref_long, ref_lat):
    delta_long = long - ref_long
    delta_lat = lat - ref_lat
    delta_long_corr = delta_long * np.cos(np.radians(ref_lat))
    return ((delta_long_corr)**2 +(delta_lat)**2)**(1/2)*2*np.pi*6378/360

class sqftPriceCalculator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['sqft_price'] = (X.price/(X.sqft_living + X.sqft_lot)).round(2)                       
        return X
    
class centerDistanceCalculator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['delta_lat'] = np.absolute(47.62774- X['lat'])
        X['delta_long'] = np.absolute(-122.24194-X['long'])
        X['center_distance']= ((X['delta_long']* np.cos(np.radians(47.6219)))**2 
                                    + X['delta_lat']**2)**(1/2)*2*np.pi*6378/360                       
        return X

class waterfrontCalculator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        water_list= X.query('waterfront == 1')
        water_distance = []
        for idx, lat in X.lat.items():
            ref_list = []
            for x,y in zip(list(water_list.long), list(water_list.lat)):
                ref_list.append(distance_calculater(X.long[idx], X.lat[idx],x,y).min())
            water_distance.append(min(ref_list))
        X['water_distance'] = water_distance
        return X

data_engineering_pipeline = Pipeline([
    ('sqft_Price_Calculator', sqftPriceCalculator()),
    ('center_Distance_Calculator', centerDistanceCalculator()),
    ('waterfront_Calculator', waterfrontCalculator())
     ]
)
house_prices_en_pipeline = data_engineering_pipeline.fit_transform(house_prices_cl_pipeline)

house_prices_en_pipeline.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,last_known_change,sqft_price,delta_lat,delta_long,center_distance,water_distance
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,...,47.5112,-122.257,1340,5650,1955,32.49,0.11654,0.01506,13.022012,0.678977
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,47.7210,-122.319,1690,7639,1991,54.83,0.09326,0.07706,11.882906,2.910551
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,47.7379,-122.233,2720,8062,1933,16.71,0.11016,0.00894,12.281023,2.327626
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,47.5208,-122.393,1360,5000,1965,86.78,0.10694,0.15106,16.436889,0.467532
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,47.6168,-122.045,1800,7503,1987,52.25,0.01094,0.19694,14.826499,1.726771


In [35]:
data_engineering_pipeline

Pipeline(steps=[('sqft_Price_Calculator', sqftPriceCalculator()),
                ('center_Distance_Calculator', centerDistanceCalculator()),
                ('waterfront_Calculator', waterfrontCalculator())])

In [36]:
preprocessor_pipe = Pipeline([
    ('data_cleaning', data_cleaning_pipeline),
    ('data_engineering', data_engineering_pipeline),
])

In [20]:
preprocessor_pipe

Pipeline(steps=[('data_cleaning',
                 Pipeline(steps=[('remove_outlier', removeOutlier()),
                                 ('sqft_basement_transformer',
                                  sqftBasementTransformer()),
                                 ('view_transformer', viewTransformer()),
                                 ('waterfront_transformer',
                                  waterfrontTransformer()),
                                 ('last_change_transformer',
                                  lastChangeTransformer()),
                                 ('date_transformer', dateTransformer())])),
                ('data_engineering',
                 Pipeline(steps=[('sqft_Price_Calculator',
                                  sqftPriceCalculator()),
                                 ('center_Distance_Calculator',
                                  centerDistanceCalculator()),
                                 ('waterfront_Calculator',
                                  waterfrontCalculator())]))])

In [37]:
house_prices_sklearn = house_prices_data()
house_prices_sklearn = preprocessor_pipe.fit_transform(house_prices_sklearn)
house_prices_sklearn

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,last_known_change,sqft_price,delta_lat,delta_long,center_distance,water_distance
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,...,47.5112,-122.257,1340,5650,1955,32.49,0.11654,0.01506,13.022012,0.678977
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,47.7210,-122.319,1690,7639,1991,54.83,0.09326,0.07706,11.882906,2.910551
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,47.7379,-122.233,2720,8062,1933,16.71,0.11016,0.00894,12.281023,2.327626
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,47.5208,-122.393,1360,5000,1965,86.78,0.10694,0.15106,16.436889,0.467532
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,47.6168,-122.045,1800,7503,1987,52.25,0.01094,0.19694,14.826499,1.726771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,47.6993,-122.346,1530,1509,2009,135.29,0.07156,0.10406,11.154088,5.346761
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,47.5107,-122.362,1830,7200,2014,49.24,0.11704,0.12006,15.839476,1.724718
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,47.5944,-122.299,1020,2007,2009,169.66,0.03334,0.05706,5.665915,0.925361
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,0.0,0.0,...,47.5345,-122.069,1410,1287,2004,100.30,0.09324,0.17294,16.616144,2.402901


In [38]:
house_prices_sklearn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21596 entries, 0 to 21596
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 21596 non-null  int64         
 1   date               21596 non-null  datetime64[ns]
 2   price              21596 non-null  float64       
 3   bedrooms           21596 non-null  int64         
 4   bathrooms          21596 non-null  float64       
 5   sqft_living        21596 non-null  int64         
 6   sqft_lot           21596 non-null  int64         
 7   floors             21596 non-null  float64       
 8   waterfront         21596 non-null  float64       
 9   view               21596 non-null  float64       
 10  condition          21596 non-null  int64         
 11  grade              21596 non-null  int64         
 12  sqft_above         21596 non-null  int64         
 13  sqft_basement      21596 non-null  float64       
 14  zipcode    

In [30]:
print(data_validation(house_prices_sklearn, DataValidation))

               id       date     price  bedrooms  bathrooms  sqft_living  \
0      7129300520 2014-10-13  221900.0         3       1.00         1180   
1      6414100192 2014-12-09  538000.0         3       2.25         2570   
2      5631500400 2015-02-25  180000.0         2       1.00          770   
3      2487200875 2014-12-09  604000.0         4       3.00         1960   
4      1954400510 2015-02-18  510000.0         3       2.00         1680   
...           ...        ...       ...       ...        ...          ...   
21592   263000018 2014-05-21  360000.0         3       2.50         1530   
21593  6600060120 2015-02-23  400000.0         4       2.50         2310   
21594  1523300141 2014-06-23  402101.0         2       0.75         1020   
21595   291310100 2015-01-16  400000.0         3       2.50         1600   
21596  1523300157 2014-10-15  325000.0         2       0.75         1020   

       sqft_lot  floors  waterfront  view  ...      lat     long  \
0          5650    

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime
from pydantic import BaseModel
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

def house_prices_data():
    open_house_prices = pd.read_csv("../data/King_County_House_prices_dataset.csv")
    open_house_prices_data = open_house_prices.copy()
    return open_house_prices_data

class DataValidation(BaseModel):
    id: int
    date: datetime
    price: float 
    bedrooms: int 
    bathrooms: float  
    sqft_living: int 
    sqft_lot: int 
    floors: float
    waterfront: float
    view: float 
    condition: int 
    grade: int  
    sqft_above: int 
    sqft_basement: float
    zipcode: int
    lat: float 
    long: float  
    sqft_living15: int 
    sqft_lot15: int 
    last_known_change: int
    sqft_price: float
    delta_lat: float
    delta_long: float
    center_distance: float
    water_distance: float

def data_validation(df: pd.DataFrame, data_schema) -> pd.DataFrame:
    class DataframeValidation(BaseModel):
        df_as_dict: list[data_schema]
    df_as_dict = df.to_dict(orient='records')
    DataframeValidation(df_as_dict=df_as_dict)
    return df 

class removeOutlier(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X[(X['bedrooms'] / X['bathrooms']) <= 10]                                
        return X

class sqftBasementTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        X['sqft_basement'] = X['sqft_basement'].replace('?', np.NaN)
        X['sqft_basement'] = X['sqft_basement'].astype(float)
        X.eval('sqft_basement = sqft_living - sqft_above', inplace=True)                               
        return X

class viewTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['view'].fillna(0, inplace=True)                             
        return X

class waterfrontTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X.waterfront.fillna(0, inplace=True)                             
        return X

class lastChangeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        last_known_change = []
        for idx, yr_re in X.yr_renovated.items():
            if str(yr_re) == 'nan' or yr_re == 0.0:
                last_known_change.append(X.yr_built[idx])
            else:
                last_known_change.append(int(yr_re))
        X['last_known_change'] = last_known_change
        X.drop('yr_renovated', axis=1, inplace=True)
        X.drop("yr_built", axis=1, inplace=True)
        return X

class dateTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['date'] = pd.to_datetime(X['date'])#, format='%Y/%m/%d').astype('object')                       
        return X

# Data Engineering
def distance_calculater(long, lat, ref_long, ref_lat):
    delta_long = long - ref_long
    delta_lat = lat - ref_lat
    delta_long_corr = delta_long * np.cos(np.radians(ref_lat))
    return ((delta_long_corr)**2 +(delta_lat)**2)**(1/2)*2*np.pi*6378/360

class sqftPriceCalculator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['sqft_price'] = (X.price/(X.sqft_living + X.sqft_lot)).round(2)                       
        return X
    
class centerDistanceCalculator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['delta_lat'] = np.absolute(47.62774- X['lat'])
        X['delta_long'] = np.absolute(-122.24194-X['long'])
        X['center_distance']= ((X['delta_long']* np.cos(np.radians(47.6219)))**2 
                                    + X['delta_lat']**2)**(1/2)*2*np.pi*6378/360                       
        return X

class waterfrontCalculator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        water_list= X.query('waterfront == 1')
        water_distance = []
        for idx, lat in X.lat.items():
            ref_list = []
            for x,y in zip(list(water_list.long), list(water_list.lat)):
                ref_list.append(distance_calculater(X.long[idx], X.lat[idx],x,y).min())
            water_distance.append(min(ref_list))
        X['water_distance'] = water_distance
        return X

def pipelines_for_cleaning_engineering():
    data_cleaning_pipeline = Pipeline([
        ('remove_outlier', removeOutlier()),
        ('sqft_basement_transformer', sqftBasementTransformer()),
        ('view_transformer', viewTransformer()),
        ('waterfront_transformer', waterfrontTransformer()),
        ('last_change_transformer', lastChangeTransformer()),
        ('date_transformer', dateTransformer())
        ])

    data_engineering_pipeline = Pipeline([
        ('sqft_Price_Calculator', sqftPriceCalculator()),
        ('center_Distance_Calculator', centerDistanceCalculator()),
        ('waterfront_Calculator', waterfrontCalculator())
        ])

    preprocessor_pipe = Pipeline([
        ('data_cleaning', data_cleaning_pipeline),
        ('data_engineering', data_engineering_pipeline)
        ])
    return preprocessor_pipe

def clean_data():
    house_prices_sklearn = house_prices_data()
    preprocessor_pipe = pipelines_for_cleaning_engineering()
    clean_data = preprocessor_pipe.fit_transform(house_prices_sklearn)
    return clean_data

def validated_data():
    validation = clean_data()
    validated_data = data_validation(validation, DataValidation)
    return validated_data


df = validated_data()
print(df)

               id       date     price  bedrooms  bathrooms  sqft_living  \
0      7129300520 2014-10-13  221900.0         3       1.00         1180   
1      6414100192 2014-12-09  538000.0         3       2.25         2570   
2      5631500400 2015-02-25  180000.0         2       1.00          770   
3      2487200875 2014-12-09  604000.0         4       3.00         1960   
4      1954400510 2015-02-18  510000.0         3       2.00         1680   
...           ...        ...       ...       ...        ...          ...   
21592   263000018 2014-05-21  360000.0         3       2.50         1530   
21593  6600060120 2015-02-23  400000.0         4       2.50         2310   
21594  1523300141 2014-06-23  402101.0         2       0.75         1020   
21595   291310100 2015-01-16  400000.0         3       2.50         1600   
21596  1523300157 2014-10-15  325000.0         2       0.75         1020   

       sqft_lot  floors  waterfront  view  ...      lat     long  \
0          5650    